In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pandas as pd
import cv2

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping

#目前问题：尝试LRN的实现；以及目前的效果都很不好，需要考虑使用数据增强，如果仍然不行的话就要换网络，试试VGG的效果
#另一个问题是使用binary_crossentropy二分类交叉熵会报错，目前只能用多分类的，不知道这个问题解决能不能提高成功率
#了解one-hot编码，应用随机增强
#原来灰度图不是只有0和1啊卧槽，我还需要进行标准化，这是个关键问题，和数据增强一起
#目前对于数据集，验证集和测试集的思路有点混乱，明明val_accruate挺高的，但是结果就很低，为什么很奇怪，我怀疑是我哪里出线错误了？？？？
#目前先尝试直接调用官方的结构进行处理，如果也不行说明不是神经网络的问题，是我的设置有问题
#图片格式要改成224，224，尝试一下————试试用现在的三维图效果是不是更好一些
#尝试把目前的框架应用一下多分类，看有没有效果
#需要整理一下目前的所有程序和文档了，有些复杂，理清明天工作需要的部分

In [2]:
alldata= np.load(file="alldata_3D.npy")
alllable=pd.read_csv('label.csv')
label=[]
for i in range(3000):
    data = alllable['label'][i]
    if data == 'no_tumor':
        #label.append(data)
        label.append(0)
    elif data == 'meningioma_tumor':
        #label.append('meningioma_tumor')
        label.append(1)
    elif data == 'glioma_tumor':
        #label.append('glioma_tumor')
        label.append(2)
    elif data == 'pituitary_tumor':
        #label.append('pituitary_tumor')
        label.append(3)
label = np.array(label)

In [3]:
class_names1 = ['no_tumor','meningioma_tumor','glioma_tumor','pituitary_tumor']

In [4]:
#按照6：2：2的结构来分成训练集，验证集和测试集
alldata = np.array(alldata, dtype="float") / 255.0 #简单做下scale处理，没有涉及到归一化的数据，从结果上看没什么价值
train_images,test_images,train_labels,test_labels=train_test_split(alldata,label,test_size=0.2,random_state=0)
train_images,val_images,train_labels,val_labels=train_test_split(train_images,train_labels,test_size=0.25,random_state=0)

In [5]:
#加入超参数和数据增强部分

train_image_generator = keras.preprocessing.image.ImageDataGenerator( rotation_range=40,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            zoom_range=0.2,
                                            horizontal_flip=True,
                                            fill_mode='nearest')
validation_image_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [6]:
alldata.shape

(3000, 224, 224, 3)

In [7]:
class LRN(keras.layers.Layer):
    def __init__(self):
        super(LRN, self).__init__()
        self.depth_radius=2
        self.bias=1
        self.alpha=1e-4
        self.beta=0.75
    def call(self,x):
        return tf.nn.lrn(x,depth_radius=self.depth_radius,
                         bias=self.bias,alpha=self.alpha,
                         beta=self.beta)

In [8]:
#conv1+pool1+lrn1-->conv2+pool2+lrn2-->conv3-->conv4-->conv5+pool5-->fc1-->fc2-->fc3 Alexnet
'''model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=96, kernel_size=(11,11),strides=4,activation='relu',padding='same',input_shape=(224,224,3)))
model.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=2))
model.add(LRN())
model.add(keras.layers.Conv2D(filters=256,kernel_size=(5,5),strides=1,activation='relu',padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=2))
model.add(LRN())
model.add(keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=1,activation='relu',padding='same'))
model.add(keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=1,activation='relu',padding='same'))
model.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=1,activation='relu',padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(4096,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(4096,activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(4,activation="softmax")) #最后一步觉得输出数据的，我怎么特么直接抄了个1000出来
model.summary()'''
#在 input_shape 中不包含数据的 batch 大小。
#model.get_config() 返回包含模型配置信息的字典。
#from_logits=True 表示 crossEntropy 层的输入是正常的张量/logits，而如果 from_logits=False，则表示输入是一个概率，通常你应该在最后一层有一些 softmax 激活

'model = keras.models.Sequential()\nmodel.add(keras.layers.Conv2D(filters=96, kernel_size=(11,11),strides=4,activation=\'relu\',padding=\'same\',input_shape=(224,224,3)))\nmodel.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=2))\nmodel.add(LRN())\nmodel.add(keras.layers.Conv2D(filters=256,kernel_size=(5,5),strides=1,activation=\'relu\',padding=\'same\'))\nmodel.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=2))\nmodel.add(LRN())\nmodel.add(keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=1,activation=\'relu\',padding=\'same\'))\nmodel.add(keras.layers.Conv2D(filters=384,kernel_size=(3,3),strides=1,activation=\'relu\',padding=\'same\'))\nmodel.add(keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=1,activation=\'relu\',padding=\'same\'))\nmodel.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=2))\nmodel.add(keras.layers.Flatten())\nmodel.add(keras.layers.Dense(4096,activation=\'relu\'))\nmodel.add(keras.layers.Dropout(0.5))\nmodel.add(keras.layers.Dense(4096,ac

In [9]:
'''model = keras.models.Sequential()  #VGG
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224,224,3)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.summary()'''

"model = keras.models.Sequential()  #VGG\nmodel.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224,224,3)))\nmodel.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))\nmodel.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))\nmodel.add(keras.layers.Dropout(0.25))\nmodel.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))\nmodel.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))\nmodel.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))\nmodel.add(keras.layers.Dropout(0.25))\nmodel.add(keras.layers.Flatten())\nmodel.add(keras.layers.Dense(256, activation='relu'))\nmodel.add(keras.layers.Dropout(0.5))\nmodel.add(keras.layers.Dense(10, activation='softmax'))\nsgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)\nmodel.summary()"

In [10]:
model = keras.models.Sequential()  #VGG16
#model.add(keras.layers.ZeroPadding2D((1,1), input_shape=(224,224,3)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu'))
#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu'))
#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu'))
#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu'))
#model.add(keras.layers.ZeroPadding2D((1,1)))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(4096, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(4096, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(4, activation='softmax'))
#sgd = keras.optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True) #momentum是动量，SGD是随机梯度算法
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 107, 107, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 128)       0

In [11]:
'''model = keras.Sequential()
covn_base=keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',
                                input_tensor=None, input_shape=(224,224,3),
                                pooling=None,
                                classes=4)
model.add(covn_base)
model.add(keras.layers.AveragePooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dense(4,activation="softmax"))
model.summary()'''

'model = keras.Sequential()\ncovn_base=keras.applications.vgg16.VGG16(include_top=False, weights=\'imagenet\',\n                                input_tensor=None, input_shape=(224,224,3),\n                                pooling=None,\n                                classes=4)\nmodel.add(covn_base)\nmodel.add(keras.layers.AveragePooling2D(pool_size=(2, 2)))\nmodel.add(keras.layers.Dense(4,activation="softmax"))\nmodel.summary()'

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)
history =model.fit(train_images, train_labels, batch_size=32, epochs=20,validation_data=(val_images, val_labels),callbacks=[reduce_lr])

Epoch 1/20
11/57 [====>.........................] - ETA: 8s - loss: 1.3847 - accuracy: 0.2500

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(test_acc)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('Lose')
plt.ylim([0, 2])
plt.legend(loc='lower right')
plt.show()
print(test_loss)